In [1]:
import pandas as pd
import numpy as np

In [2]:
# 스트링을 날짜 형식으로 가져오기

import datetime

today='2020-01-17'
datetime.datetime.strptime(today,'%Y-%m-%d')
datetime.datetime.today() # 오늘 날짜(연월일시분초)
datetime.datetime.today().strftime('%m') # 월만 추출

'01'

In [3]:
# 1. bike data 불러오기 - 각 데이터 확인(shape, info, ...)

train=pd.read_csv('data/BikeSharing/train.csv',parse_dates=['datetime'])
test=pd.read_csv('data/BikeSharing/test.csv',parse_dates=['datetime'])

train.info()
test.info()

train.shape
test.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null datetime64[ns]
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
datetime      6493 non-null datetime64[ns]
season        6493 non-null int64
holiday       6493 non-null int64
workingday    6493 non-null int64
weather       6493 non-null int64
temp          6493 non-null float64
atemp         6493 non-null float64
humidity      6493 non-null

(6493, 9)

In [ ]:
# 2. 중복 확인
# - 훈련 데이터, 테스트 데이터 중복 값 있는지 확인
# - 훈련 데이터, 테스트 데이터 null 값 확인

train.duplicated().sum()
test.duplicated().sum()

train.isnull().sum()
test.isnull().sum()

In [ ]:
train.datetime.dt.year
train.datetime.dt.month
train.datetime.dt.day
train.datetime.dt.minute
train.datetime.dt.dayofweek # 요일정보 (월요일: 0, 일요일: 6)

In [4]:
# 3. datetime에서 년,월,일,시,분,초를 추출하여 각각의 column으로 구성
# && 4. 근무일 유무/요일/시즌/날씨에 따른 시간대별 자전거 대여량 구하기
# ex) df['year]=df['datetime'].dt.year
# -년,월,일,시 에 따른 평균 대여량 구하기
# => 분석 결과 reporting (ex. 여름에 대여량이 많다)

# 날짜를 연월일시...로 나누어 각각의 열로 구성하는 함수
def dateSplit(data):
    col=['year','month','day','hour','minute','second']
    for i in col:
        data[i]=eval('data.datetime.dt.'+i)
    return data

# 열원일시... 별 평균값 출력하는 함수
# 근무일 여부, 계절, 날씨에도 적용할 수 있도록 구성
def meanCnt(data,key):
    if key=='date':
        col=['year','month','day','hour','minute','second']
    elif key=='work':
        col=['workingday']
    elif key=='season' or key=='weather':
        col=[key]
    else:
        print('Please Check Your Key')
        return
    col_new1=[] # 결과값 열(year, month, day...) 이름 저장1
    col_new2=[] # 결과값 열(2011, 2012...) 이름 저장2 (멀티인덱스)
    res=[] # 평균값 저장
    for i in col:
        kind=sorted(eval('data.'+i).drop_duplicates())
        for j in kind:
            col_new1.append(i)
            col_new2.append(int(j))
            res.append(data.loc[eval('data.'+i)==j,'count'].mean())
    col_new=[col_new1]+[col_new2]
    return pd.DataFrame(res,index=col_new,columns=['Mean Count']).T

dateSplit(train)
dateSplit(test)

meanCnt(train,'date')
meanCnt(train,'work')
meanCnt(train,'season')
meanCnt(train,'weather')

weather                              
                     1          2           3      4
Mean Count  205.236791  178.95554  118.846333  164.0

In [ ]:
# 5. 연도별/월별 자전거 대여량 구하기

# 연도별/월별 총 자전거 대여량 구하는 함수
def allCnt(data,key):
    res=[]
    kind=sorted(eval('data.'+key).drop_duplicates())
    for i in kind:
        res.append(data.loc[eval('data.'+key)==i,'count'].sum())
    return pd.DataFrame(res,index=kind,columns=['Total Count']).T

allCnt(train,'year')
allCnt(train,'month')

In [5]:
# 6. 아웃라이어제거 - 정상범위 데이터 : count열값-count열값평균 < 3*(count.std)

def delOutlier(data):
    cnts=data['count']
    data=data.loc[(cnts-cnts.mean())<3*cnts.std()]
    return data

train=delOutlier(train)

In [ ]:
# 7. 시즌컬럼 원핫 인코딩
train=pd.concat([train,pd.get_dummies(train.season,prefix='season')],axis=1)

In [6]:
# 8.데이터 보정 - 풍속 등 각 필드에 0으로 저장되어 있는 값을 근사값으로 보정

# Srep 1. 풍속과 상관관계 높은 top3 변수 추리기
def corTop3(data,sub): #(데이터셋, '풍속')
    cor=abs(data.corr().loc[sub])
    del cor['registered'] # test에 없는 변수들 삭제
    del cor['casual']
    del cor[sub] # 자기자신과의 상관관계는 무조건 1이므로 삭제
    top3=pd.DataFrame(cor.nlargest(3)).index # 습도, 월, 계절
    return top3

windTop3=corTop3(train,'windspeed')
    
    
    
# Step 2. top3 변수값 기준으로 knn 알고리즘 실행하는 함수
def tryKnn(data,sub,top): #(데이터셋, '풍속', 풍속 주요 영향 인자 리스트)
    
    # 각 영향 인자 별 데이터
    top1=data[top[0]] 
    top2=data[top[1]]
    top3=data[top[2]]
    
    # 보간 필요한 열 인덱스 선택
    idx=data.loc[(data[sub]==0)|(np.isnan(data[sub]))].index 

    for i in idx:
        # 보간 대상 열의 top3 데이터 저장
        targetTop1=top1[i] # 습도
        targetTop2=top2[i] # 월
        targetTop3=top3[i] # 계절
        
        # 가장 가까운 거리의 데이터셋 찾기 ##### 정규화 안했드아앙!!!!!
        dist1=(np.array(top1)-targetTop1)**2
        dist2=(np.array(top2)-targetTop2)**2
        dist3=(np.array(top3)-targetTop3)**2
        
        # 가장 가까운 데이터 5개의 행 인덱스 추출
        dist=pd.DataFrame(dist1+dist2+dist3,columns=['distance']).nsmallest(20,'distance').index
        # 추출된 5개 데이터 평균을 보간값으로 적용
        data.loc[i,sub]=data.loc[dist,sub].mean()
        
    return data
        
trainInt=tryKnn(train,'windspeed',windTop3)

C:\Users\student\Anaconda3\lib\site-packages\pandas\core\indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [7]:
# 9. 바이크 대여수 예상되는 값 출력

# train 데이터와 test 데이터 병합, 위의 함수 활용해 count 빈 부분 계산
wholeData=pd.concat([trainInt,test],ignore_index=True)
countTop3=corTop3(wholeData,'count')

# 원하는 부분만 파일로 저장
result=tryKnn(wholeData,'count',countTop3).loc[10739:,['datetime','count']]
result.to_csv('sub3.csv',index=False)

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [8]:
result.isnull().sum()

datetime    0
count       0
dtype: int64

In [ ]:
np.isnan(wholeData['count'])